---
description: Continual Learning Algorithms Prototyping Made Easy
---
# Training

Welcome to the "_Training_" tutorial of the "_From Zero to Hero_" series. In this part we will present the functionalities offered by the `training` module.

First, let's install Avalanche. You can skip this step if you have installed it already.

In [1]:
!pip install avalanche-lib==0.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 894.6/894.6 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.4/532.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.8/452.8 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7393 sha256=130e779868b5aa427f8fe1e0a9619c4f7c2f511f053d326e167125e9ff49c0c0
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853a

## 💪 The Training Module

The `training` module in _Avalanche_ is designed with modularity in mind. Its main goals are to:

1. provide a set of popular **continual learning baselines** that can be easily used to run experimental comparisons;
2. provide simple abstractions to **create and run your own strategy** as efficiently and easily as possible starting from a couple of basic building blocks we already prepared for you.

At the moment, the `training` module includes three main components:

* **Templates**: these are high level abstractions used as a starting point to define the actual strategies. The templates contain already implemented basic utilities and functionalities shared by a group of strategies (e.g. the `BaseSGDTemplate` contains all the implemented methods to deal with strategies based on SGD).
* **Strategies**: these are popular baselines already implemented for you which you can use for comparisons or as base classes to define a custom strategy.
* **Plugins**: these are classes that allow adding some specific behavior to your own strategy. The plugin system allows defining reusable components which can be easily combined (e.g. a replay strategy, a regularization strategy). They are also used to automatically manage logging and evaluation.

Keep in mind that many Avalanche components are independent of Avalanche strategies. If you already have your own strategy which does not use Avalanche, you can use Avalanche's benchmarks, models, data loaders, and metrics without ever looking at Avalanche's strategies!

## 📈 How to Use Strategies & Plugins

If you want to compare your strategy with other classic continual learning algorithms or baselines, in _Avalanche_ you can instantiate a strategy with a couple of lines of code.

### Strategy Instantiation
Most strategies require only 3 mandatory arguments:
- **model**: this must be a `torch.nn.Module`.
- **optimizer**: `torch.optim.Optimizer` already initialized on your `model`.
- **loss**: a loss function such as those in `torch.nn.functional`.

Additional arguments are optional and allow you to customize training (batch size, number of epochs, ...) or strategy-specific parameters (memory size, regularization strength, ...).

In [2]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.models import SimpleMLP
from avalanche.training.supervised import Naive, CWRStar, Replay, GDumb, Cumulative, LwF, GEM, AGEM, EWC  # and many more!

model = SimpleMLP(num_classes=10)
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()
cl_strategy = Naive(
    model, optimizer, criterion,
    train_mb_size=100, train_epochs=4, eval_mb_size=100
)

### Training & Evaluation

Each strategy object offers two main methods: `train` and `eval`. Both of them, accept either a _single experience_(`Experience`) or a _list of them_, for maximum flexibility.

We can train the model continually by iterating over the `train_stream` provided by the scenario.

In [3]:
from avalanche.benchmarks.classic import SplitMNIST

# scenario
benchmark = SplitMNIST(n_experiences=5, seed=1)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    results.append(cl_strategy.eval(benchmark.test_stream))

100%|██████████| 9912422/9912422 [00:00<00:00, 131357951.78it/s]


Extracting /root/.avalanche/data/mnist/TensorMNIST/raw/train-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/TensorMNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 41343240.21it/s]


Extracting /root/.avalanche/data/mnist/TensorMNIST/raw/train-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/TensorMNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 43722484.28it/s]

Extracting /root/.avalanche/data/mnist/TensorMNIST/raw/t10k-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/TensorMNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 21096931.08it/s]


Extracting /root/.avalanche/data/mnist/TensorMNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/TensorMNIST/raw

Starting experiment...
Start of experience:  0
Current Classes:  [5, 6]
-- >> Start of training phase << --
100%|██████████| 114/114 [00:02<00:00, 50.70it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.3927
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8926
100%|██████████| 114/114 [00:02<00:00, 53.02it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.1022
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9686
100%|██████████| 114/114 [00:02<00:00, 46.25it/s]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0859
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9737
100%|██████████| 114/114 [00:02<00:00, 51.09it/s]
Epoch 3 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0762
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9760
-- >> End of training phase << --
Training completed
Co

### Adding Plugins

Most continual learning strategies follow roughly the same training/evaluation loops, i.e. a simple naive strategy (a.k.a. finetuning) augmented with additional behavior to counteract catastrophic forgetting. The plugin systems in Avalanche is designed to easily augment continual learning strategies with custom behavior, without having to rewrite the training loop from scratch. Avalanche strategies accept an optional list of `plugins` that will be executed during the training/evaluation loops.

For example, early stopping is implemented as a plugin:

In [4]:
from avalanche.training.plugins import EarlyStoppingPlugin

strategy = Naive(
    model, optimizer, criterion,
    plugins=[EarlyStoppingPlugin(patience=10, val_stream_name='train')])

In Avalanche, most continual learning strategies are implemented using plugins, which makes it easy to combine them together. For example, it is extremely easy to create a hybrid strategy that combines replay and EWC together by passing the appropriate `plugins` list to the `SupervisedTemplate`:

In [5]:
from avalanche.training.templates import SupervisedTemplate
from avalanche.training.plugins import ReplayPlugin, EWCPlugin

replay = ReplayPlugin(mem_size=100)
ewc = EWCPlugin(ewc_lambda=0.001)
strategy = SupervisedTemplate(
    model, optimizer, criterion,
    plugins=[replay, ewc])

Beware that most strategy plugins modify the internal state. As a result, not all the strategy plugins can be combined together. For example, it does not make sense to use multiple replay plugins since they will try to modify the same strategy variables (mini-batches, dataloaders), and therefore they will be in conflict.

## 📝 A Look Inside Avalanche Strategies

If you arrived at this point you already know how to use Avalanche strategies and are ready to use it. However, before making your own strategies you need to understand a little bit the internal implementation of the training and evaluation loops.

In _Avalanche_ you can customize a strategy in 2 ways:

1. **Plugins**: Most strategies can be implemented as additional code that runs on top of the basic training and evaluation loops (e.g. the `Naive` strategy). Therefore, the easiest way to define a custom strategy such as a regularization or replay strategy, is to define it as a custom plugin. The advantage of plugins is that they can be combined, as long as they are compatible, i.e. they do not modify the same part of the state. The disadvantage is that in order to do so you need to understand the strategy loop, which can be a bit complex at first.
2. **Subclassing**: In _Avalanche_, continual learning strategies inherit from the appropriate template, which provides generic training and evaluation loops. The most high level template is the `BaseTemplate`, from which all the _Avalanche_'s strategies inherit. Most template's methods can be safely overridden (with some caveats that we will see later).

Keep in mind that if you already have a working continual learning strategy that does not use _Avalanche_, you can use most Avalanche components such as `benchmarks`, `evaluation`, and `models` without using _Avalanche_'s strategies!

### Training and Evaluation Loops

As we already mentioned, _Avalanche_ strategies inherit from the appropriate template (e.g. continual supervised learning strategies inherit from the `SupervisedTemplate`). These templates provide:

1. Basic _Training_ and _Evaluation_ loops which define a naive (finetuning) strategy.
2. _Callback_ points, which are used to call the plugins at a specific moments during the loop's execution.
3. A set of variables representing the state of the loops (current model, data, mini-batch, predictions, ...) which allows plugins and child classes to easily manipulate the state of the training loop.

The training loop has the following structure:
```text
train
    before_training

    before_train_dataset_adaptation
    train_dataset_adaptation
    after_train_dataset_adaptation
    make_train_dataloader
    model_adaptation
    make_optimizer
    before_training_exp  # for each exp
        before_training_epoch  # for each epoch
            before_training_iteration  # for each iteration
                before_forward
                after_forward
                before_backward
                after_backward
            after_training_iteration
            before_update
            after_update
        after_training_epoch
    after_training_exp
    after_training
```

The evaluation loop is similar:
```text
eval
    before_eval
    before_eval_dataset_adaptation
    eval_dataset_adaptation
    after_eval_dataset_adaptation
    make_eval_dataloader
    model_adaptation
    before_eval_exp  # for each exp
        eval_epoch  # we have a single epoch in evaluation mode
            before_eval_iteration  # for each iteration
                before_eval_forward
                after_eval_forward
            after_eval_iteration
    after_eval_exp
    after_eval
```

Methods starting with `before/after` are the methods responsible for calling the plugins.
Notice that before the start of each experience during training we have several phases:
- *dataset adaptation*: This is the phase where the training data can be modified by the strategy, for example by adding other samples from a separate buffer.
- *dataloader initialization*: Initialize the data loader. Many strategies (e.g. replay) use custom dataloaders to balance the data.
- *model adaptation*: Here, the dynamic models (see the `models` tutorial) are updated by calling their `adaptation` method.
- *optimizer initialization*: After the model has been updated, the optimizer should also be updated to ensure that the new parameters are optimized.

### Strategy State
The strategy state is accessible via several attributes. Most of these can be modified by plugins and subclasses:
- `self.clock`: keeps track of several event counters.
- `self.experience`: the current experience.
- `self.adapted_dataset`: the data modified by the dataset adaptation phase.
- `self.dataloader`: the current dataloader.
- `self.mbatch`: the current mini-batch. For supervised classification problems, mini-batches have the form `<x, y, t>`, where `x` is the input, `y` is the target class, and `t` is the task label.
- `self.mb_output`: the current model's output.
- `self.loss`: the current loss.
- `self.is_training`: `True` if the strategy is in training mode.

## How to Write a Plugin
Plugins provide a simple solution to define a new strategy by augmenting the behavior of another strategy (typically the `Naive` strategy). This approach reduces the overhead and code duplication, **improving code readability and prototyping speed**.

Creating a plugin is straightforward. As with strategies, you must create a class that inherits from the corresponding plugin template (`BasePlugin`, `BaseSGDPlugin`, `SupervisedPlugin`) and implements the callbacks that you need. The exact callback to use depend on the aim of your plugin. You can use the loop shown above to understand what callbacks you need to use. For example, we show below a simple replay plugin that uses `after_training_exp` to update the buffer after each training experience, and the `before_training_exp` to customize the dataloader. Notice that `before_training_exp` is executed after `make_train_dataloader`, which means that the `Naive` strategy already updated the dataloader. If we used another callback, such as `before_train_dataset_adaptation`, our dataloader would have been overwritten by the `Naive` strategy. Plugin methods always receive the `strategy` as an argument, so they can access and modify the strategy's state.

In [6]:
from avalanche.benchmarks.utils.data_loader import ReplayDataLoader
from avalanche.core import SupervisedPlugin
from avalanche.training.storage_policy import ReservoirSamplingBuffer


class ReplayP(SupervisedPlugin):

    def __init__(self, mem_size):
        """ A simple replay plugin with reservoir sampling. """
        super().__init__()
        self.buffer = ReservoirSamplingBuffer(max_size=mem_size)

    def before_training_exp(self, strategy: "SupervisedTemplate",
                            num_workers: int = 0, shuffle: bool = True,
                            **kwargs):
        """ Use a custom dataloader to combine samples from the current data and memory buffer. """
        if len(self.buffer.buffer) == 0:
            # first experience. We don't use the buffer, no need to change
            # the dataloader.
            return
        strategy.dataloader = ReplayDataLoader(
            strategy.adapted_dataset,
            self.buffer.buffer,
            oversample_small_tasks=True,
            num_workers=num_workers,
            batch_size=strategy.train_mb_size,
            shuffle=shuffle)

    def after_training_exp(self, strategy: "SupervisedTemplate", **kwargs):
        """ Update the buffer. """
        self.buffer.update(strategy, **kwargs)


benchmark = SplitMNIST(n_experiences=5, seed=1)
model = SimpleMLP(num_classes=10)
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = CrossEntropyLoss()
strategy = Naive(model=model, optimizer=optimizer, criterion=criterion, train_mb_size=128,
                 plugins=[ReplayP(mem_size=2000)])
strategy.train(benchmark.train_stream)
strategy.eval(benchmark.test_stream)

-- >> Start of training phase << --
100%|██████████| 89/89 [00:01<00:00, 44.61it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.1751
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9478
100%|██████████| 100/100 [00:02<00:00, 39.05it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.5911
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8915
100%|██████████| 92/92 [00:02<00:00, 37.44it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.5489
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8564
100%|██████████| 95/95 [00:02<00:00, 33.47it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.7076
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8095
100%|██████████| 95/95 [00:02<00:00, 38.14it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.7410
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8081
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on e

{'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.8081422245720706,
 'Loss_Epoch/train_phase/train_stream/Task000': 0.7410415130876107,
 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.7670270270270271,
 'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 0.7227041492591033,
 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001': 0.9077065066912783,
 'Loss_Exp/eval_phase/test_stream/Task000/Exp001': 0.35510920158002474,
 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002': 0.8167860798362334,
 'Loss_Exp/eval_phase/test_stream/Task000/Exp002': 0.6835261336856635,
 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp003': 0.4933135215453195,
 'Loss_Exp/eval_phase/test_stream/Task000/Exp003': 1.1685963427595363,
 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp004': 0.9786069651741294,
 'Loss_Exp/eval_phase/test_stream/Task000/Exp004': 0.07781347763858995,
 'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.7945,
 'Loss_Stream/eval_phase/test_stream/Task000': 0.5957935487260111}

The animation below shows the execution and callbacks steps of a Naive strategy that is extended with the EWC plugin:

<img align='center' style='max-width: 800px' src='https://github.com/ContinualAI/avalanche/blob/master/.gitbook/assets/ewc_template_animation.gif?raw=1'>


Check base plugin's documentation for a complete list of the available callbacks.

## How to Write a Custom Strategy

You can always define a custom strategy by overriding the corresponding template methods.
However, There is an important caveat to keep in mind. If you override a method, you must remember to call all the callback's handlers (the methods starting with `before/after`) at the appropriate points. For example, `train` calls `before_training` and `after_training` before and after the training loops, respectively. The easiest way to avoid mistakes is to start from the template's method that you want to override and modify it based on your own needs without removing the callbacks handling.

Notice that the `EvaluationPlugin` (see `evaluation` tutorial) uses the strategy callbacks.

As an example, the `SupervisedTemplate`, for continual supervised strategies, provides the global state of the loop in the strategy's attributes, which you can safely use when you override a method. For instance, the `Cumulative` strategy trains a model continually on the union of all the experiences encountered so far. To achieve this, the cumulative strategy overrides `adapt_train_dataset` and updates `self.adapted_dataset' by concatenating all the previous experiences with the current one.

In [7]:
from avalanche.benchmarks.utils import AvalancheConcatDataset
from avalanche.training.templates import SupervisedTemplate


class Cumulative(SupervisedTemplate):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.dataset = None  # cumulative dataset

    def train_dataset_adaptation(self, **kwargs):
        super().train_dataset_adaptation(**kwargs)
        curr_data = self.experience.dataset
        if self.dataset is None:
            self.dataset = curr_data
        else:
            self.dataset = AvalancheConcatDataset([self.dataset, curr_data])
        self.adapted_dataset = self.dataset.train()

strategy = Cumulative(model=model, optimizer=optimizer, criterion=criterion, train_mb_size=128)
strategy.train(benchmark.train_stream)

-- >> Start of training phase << --
100%|██████████| 89/89 [00:01<00:00, 45.46it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.1389
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9608
100%|██████████| 188/188 [00:04<00:00, 42.29it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.1352
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9643
100%|██████████| 280/280 [00:06<00:00, 45.75it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.1962
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9446
100%|██████████| 375/375 [00:08<00:00, 43.54it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2429
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9307
100%|██████████| 469/469 [00:10<00:00, 43.83it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2452
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9289
-- >> End of training phase << --


{'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.9289166666666666,
 'Loss_Epoch/train_phase/train_stream/Task000': 0.24515948289235434}

Easy, isn't it? :-\)

In general, we recommend to _implement a Strategy via plugins_, if possible. This approach is the easiest to use and requires minimal knowledge of the strategy templates. It also allows other people to re-use your plugin and facilitates interoperability among different strategies.

For example, replay strategies can be implemented as a custom strategy or as plugins. However, creating a plugin allows using the replay in conjunction with other strategies or plugins, making it possible to combine different approaches to build the ultimate continual learning algorithm!

This completes the "_Training_" chapter for the "_From Zero to Hero_" series. We hope you enjoyed it!

## 🤝 Run it on Google Colab

You can run _this chapter_ and play with it on Google Colaboratory: [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ContinualAI/avalanche/blob/master/notebooks/from-zero-to-hero-tutorial/04_training.ipynb)